In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Extraction_Data")\
        .config("spark.driver.extraClassPath", "/home/adriano/Documentos/airflow/jdbc/postgresql-42.7.4.jar")\
        .getOrCreate()

24/10/16 20:25:44 WARN Utils: Your hostname, Ubuntu-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/16 20:25:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/16 20:25:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from datetime import datetime

current_year = datetime.now().year

In [3]:
type(datetime.now().year)

int

In [4]:
ano_inicial = 2023
ano_final = 2024
for ano in range(int(ano_inicial), int(ano_final) + 1):
    print(ano)

2023
2024


In [5]:
ano_inicial = current_year
ano_final = current_year

dif_anos = (ano_final - ano_inicial) + 1
for i in range(dif_anos):
    print(ano_inicial + i)

2024


In [6]:
from pyspark.sql.functions import input_file_name, substring

origin = '/home/adriano/Documentos/airflow/database/dest/bronze/execucao-financeira'
process_all = True
ano_final = current_year
ano_inicial = current_year


if process_all == False:
    origin_file = origin + '/' + str(ano_inicial)+ '.json'
    df = spark.read.json(origin_file)
elif process_all == True:
    origin_file = origin + '/*.json'
    df = spark.read.json(origin_file) 
    df = df.withColumn('nomeArquivo',substring(input_file_name(),-9,4))


#print(df.show())
print(df.printSchema())

root
 |-- autorEmenda: string (nullable = true)
 |-- codigoAmparoLegal: long (nullable = true)
 |-- descricaoEmpenho: string (nullable = true)
 |-- fonteRecurso: string (nullable = true)
 |-- idProjetoInvestimento: string (nullable = true)
 |-- informacoesComplementares: string (nullable = true)
 |-- localEntrega: string (nullable = true)
 |-- naturezaDespesa: string (nullable = true)
 |-- nomeEsferaOrcamentaria: string (nullable = true)
 |-- nomeFavorecido: string (nullable = true)
 |-- nomeTipoEmpenho: string (nullable = true)
 |-- nrPtres: string (nullable = true)
 |-- numeroNotaEmpenhoGerada: string (nullable = true)
 |-- numeroProcesso: string (nullable = true)
 |-- pagina: long (nullable = true)
 |-- planoInterno: string (nullable = true)
 |-- planoOrcamentario: string (nullable = true)
 |-- resultadoPrimario: string (nullable = true)
 |-- tamanhoDaPagina: long (nullable = true)
 |-- tipoCredito: string (nullable = true)
 |-- ugEmitente: string (nullable = true)
 |-- ugResponsave

In [14]:
from pyspark.sql.functions import input_file_name, substring, explode, col, to_date, length
from pyspark.sql.types import ArrayType, StringType
origin = '/home/adriano/Documentos/airflow/database/dest/bronze/projeto_investimento/date'
process_all = False
ano_final = current_year
ano_inicial = current_year


if process_all == False:
    origin_file = origin + '/2024-09-18.json'
    df = spark.read.json(origin_file)
    df = df.withColumn('nomeArquivo',substring(input_file_name(),-15,10))
elif process_all == True:
    origin_file = origin + '/*.json'
    df = spark.read.json(origin_file) 
    df = df.withColumn('nomeArquivo',substring(input_file_name(),-9,4))

list_drop = ['tomadores','executores', 'teste']

df_eixos = df.drop()
df_eixos = df_eixos.withColumn("dataCadastro", to_date(col("dataCadastro"), "yyyy-MM-dd"))

drop_columns = []

for column in df.schema:
    if type(column.dataType) == ArrayType:
        drop_columns.append(column.name)

drop_columns

df_eixos = df.drop(*drop_columns)
df_eixos.printSchema()
#df.schema[5].dataType



#curs = conn.cursor()
#curs.execute(f'''delete from stg_execucao_financeira where "nomeArquivo" = '{ano_final}' ''')

#print(df.show())
#print(df.printSchema())
#df.createOrReplaceTempView("eixos")

#results = spark.sql("select idUnico, eixos.eixos.* from eixos")
#results.show()

root
 |-- cep: string (nullable = true)
 |-- dataCadastro: string (nullable = true)
 |-- dataFinalEfetiva: string (nullable = true)
 |-- dataFinalPrevista: string (nullable = true)
 |-- dataInicialEfetiva: string (nullable = true)
 |-- dataInicialPrevista: string (nullable = true)
 |-- dataSituacao: string (nullable = true)
 |-- descPlanoNacionalPoliticaVinculado: string (nullable = true)
 |-- descPopulacaoBeneficiada: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- especie: string (nullable = true)
 |-- funcaoSocial: string (nullable = true)
 |-- idUnico: string (nullable = true)
 |-- isModeladaPorBim: boolean (nullable = true)
 |-- metaGlobal: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- naturezaOutras: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- observacoesPertinentes: string (nullable = true)
 |-- populacaoBeneficiada: string (nullable = true)
 |-- qdtEmpregosGerados: strin

In [15]:
import jaydebeapi
import os
from dotenv import load_dotenv
from pathlib import Path
from datetime import timedelta

dotenv_path = Path('/home/adriano/Documentos/airflow/.env')
load_dotenv(dotenv_path=dotenv_path)

user_dw = os.getenv('USER_DW')
password_dw = os.getenv('PASSWORD_DW')
host_dw = os.getenv('HOST_DW')
database_dw = os.getenv('DATABASE_DW')
port_dw = os.getenv('PORT_DW')
driver = "org.postgresql.Driver"
mode = 'append'
properties = {"user": user_dw, "password": password_dw, "driver": driver}
url = f'jdbc:postgresql://{host_dw}:{port_dw}/{database_dw}'

drop_columns = []
for column in df.schema:
    if type(column.dataType) == ArrayType:
        drop_columns.append(column.name)
    if type(column.dataType) == StringType:
        print(df.withColumn("len_col",length(col(column.name))).groupby().max("len_col").show())

df_projeto_investimento = df.drop(*drop_columns)
df_projeto_investimento = df_projeto_investimento.withColumn("dataCadastro", to_date(col("dataCadastro"), "yyyy-MM-dd"))
df_projeto_investimento = df_projeto_investimento.withColumn("dataInicialPrevista", to_date(col("dataInicialPrevista"), "yyyy-MM-dd"))
df_projeto_investimento = df_projeto_investimento.withColumn("dataFinalPrevista", to_date(col("dataFinalPrevista"), "yyyy-MM-dd"))
df_projeto_investimento = df_projeto_investimento.withColumn("dataInicialEfetiva", to_date(col("dataInicialEfetiva"), "yyyy-MM-dd"))
df_projeto_investimento = df_projeto_investimento.withColumn("dataFinalEfetiva", to_date(col("dataFinalEfetiva"), "yyyy-MM-dd"))
df_projeto_investimento = df_projeto_investimento.withColumn("dataCadastro", to_date(col("dataCadastro"), "yyyy-MM-dd"))
df_projeto_investimento = df_projeto_investimento.withColumn("dataSituacao", to_date(col("dataSituacao"), "yyyy-MM-dd"))
df_projeto_investimento.write.jdbc(url=url, table='stg_projeto_investimento', mode=mode, properties=properties)

today = datetime.now()
date_before = today - timedelta(days=30)

mode = 'append'
url = f'jdbc:postgresql://{host_dw}:{port_dw}/{database_dw}'
properties = {"user": user_dw, "password": password_dw, "driver": driver}
conn = jaydebeapi.connect(driver, url, [user_dw, password_dw], '/home/adriano/Documentos/airflow/jdbc/postgresql-42.7.4.jar')
curs = conn.cursor()
curs.execute(f'''delete from stg_projeto_investimento_eixos         where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento_tomadores     where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento_executores    where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento_repassadores  where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento_tipos         where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento_sub_tipos     where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento_geometria     where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento_fontes_de_recurso where "datacadastro" between '{date_before}' and '{today}';
                 delete from stg_projeto_investimento               where "datacadastro" between '{date_before}' and '{today}';''')

#mode = 'append'
#properties = {"user": user_dw, "password": password_dw, "driver": driver}
#df_eixos.write.jdbc(url=url, table='stg_projeto_investimento_fontes_de_recurso', mode=mode, properties=properties)

+------------+
|max(len_col)|
+------------+
|          10|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          10|
+------------+

None
+------------+
|max(len_col)|
+------------+
|        NULL|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          10|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          10|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          10|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          10|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          53|
+------------+

None
+------------+
|max(len_col)|
+------------+
|         357|
+------------+

None
+------------+
|max(len_col)|
+------------+
|        1011|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          70|
+------------+

None
+------------+
|max(len_col)|
+------------+
|          11|
+------------+

None
+------------+
|max(len_col)

24/10/16 20:36:41 ERROR Executor: Exception in task 0.0 in stage 107.0 (TID 75)
java.sql.BatchUpdateException: Batch entry 0 INSERT INTO stg_projeto_investimento ("cep","datacadastro","datafinalefetiva","datafinalprevista","datainicialefetiva","datainicialprevista","datasituacao","descplanonacionalpoliticavinculado","descpopulacaobeneficiada","descricao","endereco","especie","funcaosocial","idunico","ismodeladaporbim","metaglobal","natureza","naturezaoutras","nome","observacoespertinentes","populacaobeneficiada","qdtempregosgerados","situacao","uf","nomearquivo") VALUES ((NULL),('2024-09-18 -03'::date),(NULL),('2027-12-31 -03'::date),(NULL),('2024-12-31 -03'::date),('2024-09-18 -03'::date),('Politica NAcional deDesenvolvimento Regional - PNDR'),('Familias residentes no perimento urbano, em area de vunerabilidade social, carente que fazem parte do cadastro único em sua maioria familias de baixa renda'),('Execução de obras de Pavimentação no município de Maribondo/Al.'),(NULL),('Construç

Py4JJavaError: An error occurred while calling o574.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 107.0 failed 1 times, most recent failure: Lost task 0.0 in stage 107.0 (TID 75) (10.0.2.15 executor driver): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO stg_projeto_investimento ("cep","datacadastro","datafinalefetiva","datafinalprevista","datainicialefetiva","datainicialprevista","datasituacao","descplanonacionalpoliticavinculado","descpopulacaobeneficiada","descricao","endereco","especie","funcaosocial","idunico","ismodeladaporbim","metaglobal","natureza","naturezaoutras","nome","observacoespertinentes","populacaobeneficiada","qdtempregosgerados","situacao","uf","nomearquivo") VALUES ((NULL),('2024-09-18 -03'::date),(NULL),('2027-12-31 -03'::date),(NULL),('2024-12-31 -03'::date),('2024-09-18 -03'::date),('Politica NAcional deDesenvolvimento Regional - PNDR'),('Familias residentes no perimento urbano, em area de vunerabilidade soc

In [44]:
date_before.isoformat()

'2024-09-13T13:31:02.426132'

In [7]:
import jaydebeapi
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('/home/adriano/Documentos/airflow/.env')
load_dotenv(dotenv_path=dotenv_path)

user_dw = os.getenv('USER_DW')
password_dw = os.getenv('PASSWORD_DW')
host_dw = os.getenv('HOST_DW')
database_dw = os.getenv('DATABASE_DW')
port_dw = os.getenv('PORT_DW')
driver = "org.postgresql.Driver"

mode = 'append'
url = f'jdbc:postgresql://{host_dw}:{port_dw}/{database_dw}'
properties = {"user": user_dw, "password": password_dw, "driver": driver}
#conn = jaydebeapi.connect(driver, url, [user_dw, password_dw], '/home/adriano/Documentos/airflow/jdbc/postgresql-42.7.4.jar')
#df.write.jdbc(url=url, table='stg_execucao_financeira', mode=mode, properties=properties)





In [8]:
#curs = conn.cursor()
#curs.execute(f'''delete from stg_execucao_financeira where "nomeArquivo" = '{ano_final}' ''')

In [10]:
import requests
import time
import json

def extract_data_api(url_base, endpoint, uf, dest_path, page_size = 100, errors_limit = -1, errors_consecutives_limit = 5, executions_limit = 200):
    success = False
    page = 0
    errors_consecutives = 0
    errors = 0
    executions = 0
    method = "GET"
    dest_path_file = dest_path + '/' + str(uf) + '.json'

    Path(dest_path).mkdir(parents=True, exist_ok=True)

    if os.path.isfile(dest_path_file) == True:
        os.remove(dest_path_file)

    content_all = []

    while success == False and (errors_consecutives < errors_consecutives_limit or errors_consecutives_limit == -1) and (errors < errors_limit or errors_limit == -1) and (executions < executions_limit or executions_limit == -1):
        url = url_base + endpoint + '?' + 'pagina=' + str(page) + '&' + 'tamanhoDaPagina=' + str(page_size) + '&' + 'uf=' + str(uf)
        response = requests.request(method, url)
        if response.status_code == 200:
            if(len(response.json()["content"]) == 0):
                success = True 
            else:
                page += 1
                errors_consecutives = 0
                executions += 1
                content_all += response.json()["content"]
                #with open(dest_path_file, 'w') as f:
                #    json.dump(response.json()["content"],f)
        else:
            errors_consecutives += 1
            errors += 1
            executions += 1
            if response.status_code == 429:
                time.sleep(1)
        print(f'Status Code: {response.status_code}\n'
            f'Executions: {executions}\n'
            f'Pages: {page}\n' 
            f'N° Registers: {len(content_all)}\n'
            f'Errors: {errors}\n'
            f'Errors Consecutives: {errors_consecutives}\n')
        time.sleep(1)

    if success == True:
        print('Execution Finished with success!')
    else:
        print('Execution Finished with error!')
        if errors_consecutives < errors_consecutives_limit:
            raise Exception("Number of consecutives errors exceeded")
        elif errors < errors_limit:
            raise Exception("Number of total errors exceeded")
        elif executions < executions_limit:
            raise Exception("Number of executions exceeded")

    with open(dest_path_file, 'w', encoding='utf-8') as f:
        json.dump(content_all, f)
    f.close()


page_size = 100
url_base = "https://api.obrasgov.gestao.gov.br"
endpoint = "/obrasgov/api/projeto-investimento"
current_year = datetime.now().year
initial_year = int(current_year)
final_year = int(current_year)
dest_path = '/home/adriano/Documentos/airflow/database/dest/bronze/execucao-financeira'
errors_limit = - 1
errors_consecutives_limit = 5
executions_limit = -1

extract_data_api(url_base, endpoint, 'PR', dest_path, page_size, errors_limit, errors_consecutives_limit, executions_limit)

Status Code: 500
Executions: 1
Pages: 0
N° Registers: 0
Errors: 1
Errors Consecutives: 1

Status Code: 200
Executions: 2
Pages: 1
N° Registers: 99
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 3
Pages: 2
N° Registers: 198
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 4
Pages: 3
N° Registers: 298
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 5
Pages: 4
N° Registers: 398
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 6
Pages: 5
N° Registers: 497
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 7
Pages: 6
N° Registers: 597
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 8
Pages: 7
N° Registers: 696
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 9
Pages: 8
N° Registers: 796
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 10
Pages: 9
N° Registers: 896
Errors: 1
Errors Consecutives: 0

Status Code: 200
Executions: 11
Pages: 10
N° Registers: 994
Errors: 1
Errors Conse

In [11]:
from datetime import datetime, date, timedelta
today = datetime.today()
days_before = today - timedelta(days=2)


date_generated = [days_before + timedelta(days=x) for x in range(0, (today-days_before).days)]

for d in date_generated:
    print(d)

2024-10-11 10:22:48.977500
2024-10-12 10:22:48.977500


In [ ]:
def extract_api(url_base, endpoint, param, method, path_dest_env, fun_api, page_size, errors_limit, errors_consecutives_limit, executions_limit):
    success = False
    page = 0
    errors_consecutives = 0
    errors = 0
    executions = 0
    dest_path = os.getenv(path_dest_env)
    dest_path_file = dest_path + '/' + str(param) + '.json'

    Path(dest_path).mkdir(parents=True, exist_ok=True)

    if os.path.isfile(dest_path_file) == True:
        os.remove(dest_path_file)

    content_all = []

    while success == False and (errors_consecutives < errors_consecutives_limit or errors_consecutives_limit == -1) and (errors < errors_limit or errors_limit == -1) and (executions < executions_limit or executions_limit == -1):
        success, content,  page, errors_consecutives, errors, executions, status_code =  fun_api(url_base, endpoint, param, method, page_size, page, errors_consecutives, errors, executions)

        if content is not None:
            content_all.append(content)

        print(f'Status Code: {status_code}\n'
            f'Executions: {executions}\n'
            f'Pages: {page}\n' 
            f'N° Registers: {len(content_all)}\n'
            f'Errors: {errors}\n'
            f'Errors Consecutives: {errors_consecutives}\n')
        time.sleep(1)

    if success == True:
        print('Execution Finished with success!')
    else:
        print('Execution Finished with error!')
        if errors_consecutives < errors_consecutives_limit:
            raise Exception("Number of consecutives errors exceeded")
        elif errors < errors_limit:
            raise Exception("Number of total errors exceeded")
        elif executions < executions_limit:
            raise Exception("Number of executions exceeded")

    with open(dest_path_file, 'w', encoding='utf-8') as f:
        json.dump(content_all, f)
    f.close()

def extract_api_projeto_investimento_uf(url_base, endpoint, param, method, page_size, page, errors_consecutives, errors, executions):
    params = {"pagina": str(page), "tamanhoDaPagina": str(page_size), "uf": str(param)}
    url = generate_url(url_base, endpoint, params)
    response = requests.request(method, url)
    if response.status_code == 200:
        if(len(response.json()["content"]) == 0):
            success = True
            return success, None,  page, errors_consecutives, errors, executions, response.status_code
        else:
            page += 1
            errors_consecutives = 0
            executions += 1
            content = response.json()["content"]
            return success, content,  page, errors_consecutives, errors, executions, response.status_code
    else:
        errors_consecutives += 1
        errors += 1
        executions += 1
        if response.status_code == 429:
            time.sleep(1)
        return success, None,  page, errors_consecutives, errors, executions, response.status_code

@task()
def extract_data_api_project(url_base, endpoint, date, page_size = 100, errors_limit = -1, errors_consecutives_limit = 5, executions_limit = 200):
    method = "GET"
    path_dest_env = "PATH_DEST_PROJETO_INVESTIMENTO"
    extract_api(url_base, endpoint, date, method, path_dest_env, extract_api_projeto_investimento_uf, page_size, errors_limit, errors_consecutives_limit, executions_limit)

def extract_api_execucao_financeira_year(url_base, endpoint, year, method, page_size, page, errors_consecutives, errors, executions):
    params = {"pagina": str(page), "tamanhoDaPagina": str(page_size), "anoFinal": str(year), "anoInicial": str(year)}
    url = generate_url(url_base, endpoint, params)
    success = False
    response = requests.request(method, url)
    if response.status_code == 200:
        page += 1
        errors_consecutives = 0
        executions += 1
        content = response.json()["content"]
        return success, content,  page, errors_consecutives, errors, executions, response.status_code
    elif response.status_code == 404:
        success = True
        return success, None,  page, errors_consecutives, errors, executions, response.status_code 
    else:
        errors_consecutives += 1
        errors += 1
        executions += 1
        if response.status_code == 429:
            time.sleep(1)
        return success, None,  page, errors_consecutives, errors, executions, response.status_code

In [15]:
def generate_dates_reprocessing(date, days):
    days_date = []
    date_before = date - timedelta(days=days)
    date_generated = [date_before + timedelta(days=x) for x in range(0, (date-date_before).days)]

    for d in date_generated:
        days_date.append(str(d.strftime('%Y-%m-%d')))
        print(d.strftime('%Y-%m-%d'))
    return days_date

generate_dates_reprocessing(today, 30)

2024-09-13
2024-09-14
2024-09-15
2024-09-16
2024-09-17
2024-09-18
2024-09-19
2024-09-20
2024-09-21
2024-09-22
2024-09-23
2024-09-24
2024-09-25
2024-09-26
2024-09-27
2024-09-28
2024-09-29
2024-09-30
2024-10-01
2024-10-02
2024-10-03
2024-10-04
2024-10-05
2024-10-06
2024-10-07
2024-10-08
2024-10-09
2024-10-10
2024-10-11
2024-10-12


['2024-09-13',
 '2024-09-14',
 '2024-09-15',
 '2024-09-16',
 '2024-09-17',
 '2024-09-18',
 '2024-09-19',
 '2024-09-20',
 '2024-09-21',
 '2024-09-22',
 '2024-09-23',
 '2024-09-24',
 '2024-09-25',
 '2024-09-26',
 '2024-09-27',
 '2024-09-28',
 '2024-09-29',
 '2024-09-30',
 '2024-10-01',
 '2024-10-02',
 '2024-10-03',
 '2024-10-04',
 '2024-10-05',
 '2024-10-06',
 '2024-10-07',
 '2024-10-08',
 '2024-10-09',
 '2024-10-10',
 '2024-10-11',
 '2024-10-12']